In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pickle

In [2]:
from geopy import distance
from sklearn.linear_model import LinearRegression

In [3]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
import math
from copy import deepcopy

In [4]:
df = pd.read_csv("C4K.csv")
#zip_df = pd.read_csv("DemographicData.csv")

In [5]:
np.array(df["ZIP CODE"])

array([ 70311.,  73002.,  73003.,  73005.,  73008.,  73011.,  73012.,
        73013.,  73015.,  73016.,  73017.,  73018.,  73019.,  73020.,
        73028.,  73036.,  73044.,  73045.,  73049.,  73050.,  73051.,
        73052.,  73054.,  73055.,  73059.,  73064.,  73065.,  73069.,
        73071.,  73072.,  73078.,  73084.,  73090.,  73096.,  73099.,
        73101.,  73102.,  73103.,  73104.,  73105.,  73106.,  73107.,
        73108.,  73109.,  73110.,  73111.,  73112.,  73113.,  73114.,
        73115.,  73116.,  73117.,  73118.,  73119.,  73120.,  73121.,
        73122.,  73127.,  73128.,  73129.,  73130.,  73132.,  73134.,
        73135.,  73137.,  73139.,  73140.,  73141.,  73142.,  73145.,
        73149.,  73150.,  73159.,  73160.,  73162.,  73166.,  73170.,
        73173.,  73179.,  73401.,  73505.,  73507.,  73658.,  73701.,
        73772.,  74011.,  74058.,  74107.,  74331.,  74426.,  74436.,
        74437.,  74441.,  74455.,  74501.,  74525.,  74578.,  74728.,
        74743.,  748

In [6]:
zip_shapes = gpd.read_file("zipShapes/oklahoma-zip-code-boundaries.shp")

In [7]:
float_items = []

In [8]:
for item in zip_shapes["zcta5ce00"]:
    try: 
        float_items.append(float(item))
    except:
        pass

In [9]:
non_code= [element for element in float_items if element not in np.array(df["ZIP CODE"])] 

In [10]:
print(len(non_code))

376


In [11]:
df = df.drop("Unnamed: 0", axis = 1)

In [12]:
date_array = []
for number in np.unique(df["DATE"]):
    date_array += list(np.full(len(non_code), int(number)))

In [13]:
#date_array = np.concat([np.full(len(non_code), 2018),np.full(len(non_code), 2019),np.full(len(non_code), 2020),np.full(len(non_code), 2021), np.full(len(non_code), 2022),np.full(len(non_code), 2023)], axis = 0)

In [14]:
filled_df = pd.DataFrame(list(zip(np.tile(non_code,len(np.unique(df["DATE"]))),np.zeros(len(non_code)*len(np.unique(df["DATE"]))), date_array)),columns = ["ZIP CODE", "Given", "DATE"])

In [15]:
filled_df

,ZIP CODE,Given,DATE
0,74761.0,0.0,2018
1,74738.0,0.0,2018
2,74764.0,0.0,2018
3,74766.0,0.0,2018
4,73548.0,0.0,2018
...,...,...,...
2251,74842.0,0.0,2023
2252,73661.0,0.0,2023
2253,73664.0,0.0,2023
2254,73841.0,0.0,2023


In [16]:
df = pd.concat([filled_df, df], axis = 0)

In [17]:
df = df.set_index("ZIP CODE")

In [18]:
df.index = np.array(df.index).astype('U5').astype(str)
df.index.name = "ZIP CODE"

In [19]:
df = df[["DATE", "Given"]]

In [20]:
df["DATE"]= df["DATE"] - 2018

In [21]:
indexes = list(set(df.index))

In [22]:
combined_df = None

In [23]:
for index in indexes:
    test_df = df.loc[index]
    # manipulate these areas, rotate on DATE
    
    if type(test_df) == pd.core.series.Series:
        test_df = pd.DataFrame(test_df).transpose()
   
    placeholder_dict = {}
    # year you wish to predict (2018 + 6) = prediction for 2024
    for j in range(5):
        if len(test_df.loc[test_df["DATE"] == j]) > 1:
            placeholder_dict[j] = [float(test_df.loc[test_df["DATE"] == j]["Given"].sum())]
        else:
            try:
                placeholder_dict[j] = [(test_df.loc[test_df["DATE"] == j]["Given"].values)[0]]
            except: 
                placeholder_dict[j] = [0]
    final_df  = pd.DataFrame.from_dict(placeholder_dict)
    final_df.index = [index]
                
    if combined_df is None:
        combined_df = final_df
    else:
        combined_df = pd.concat([combined_df, final_df], axis = 0)

In [24]:
combined_df

,0,1,2,3,4
74549,0.0,0.0,0.0,0.0,0.0
73847,0.0,0.0,0.0,0.0,0.0
73554,0.0,0.0,0.0,0.0,0.0
74748,0.0,0.0,0.0,0.0,0.0
73104,19.0,40.0,9.0,108.0,92.0
...,...,...,...,...,...
74759,0.0,0.0,0.0,0.0,0.0
74051,0.0,0.0,0.0,0.0,0.0
73735,0.0,0.0,0.0,91.0,0.0
73722,0.0,0.0,0.0,0.0,52.0


In [25]:
combined_df = combined_df.replace(0, .1)

In [26]:
combined_df

,0,1,2,3,4
74549,0.1,0.1,0.1,0.1,0.1
73847,0.1,0.1,0.1,0.1,0.1
73554,0.1,0.1,0.1,0.1,0.1
74748,0.1,0.1,0.1,0.1,0.1
73104,19.0,40.0,9.0,108.0,92.0
...,...,...,...,...,...
74759,0.1,0.1,0.1,0.1,0.1
74051,0.1,0.1,0.1,0.1,0.1
73735,0.1,0.1,0.1,91.0,0.1
73722,0.1,0.1,0.1,0.1,52.0


In [27]:
def best_regression_model(K, Y):
    # Reshape X to be 2D
    reshape_X = K.reshape(-1, 1)
    # Initialize variables to store the best model info
    best_r2 = -np.inf
    best_equation = None
    
    # Linear Regression
    linear_model = LinearRegression()
    linear_model.fit(reshape_X, Y)
    y_pred_linear = linear_model.predict(reshape_X)
    r2_linear = r2_score(Y, y_pred_linear)
    if r2_linear > best_r2:
        best_r2 = r2_linear
        best_equation = (lambda x: (linear_model.coef_[0]*x + linear_model.intercept_), best_r2)
    
    # Quadratic Regression
    quadratic_transform = PolynomialFeatures(degree=2)
    X_quad = quadratic_transform.fit_transform(reshape_X)
    quad_model = LinearRegression()
    quad_model.fit(X_quad, Y)
    y_pred_quad = quad_model.predict(X_quad)
    r2_quad = r2_score(Y, y_pred_quad)
    if r2_quad > best_r2:
        best_r2 = r2_quad
        best_equation = (lambda x: (quad_model.coef_[2]*(x*x) + quad_model.coef_[1] * x + quad_model.intercept_), best_r2)
    
    # Exponential Regression (y = a * exp(b * x))
    Y_log = np.log(Y)
    exp_model = LinearRegression()
    exp_model.fit(reshape_X, Y_log)
    y_pred_exp = np.exp(exp_model.predict(reshape_X))
    r2_exp = r2_score(Y, y_pred_exp)
    if r2_exp > best_r2:
        best_r2 = r2_exp
        best_equation =  (lambda x: (math.exp(exp_model.intercept_) * math.exp(exp_model.coef_[0]*x)), best_r2)
    
    # Logarithmic Regression (y = a * log(x) + b)
    X_log = np.log(K.astype(float)).reshape(-1,1)
    X_log[0] = -100
    log_model = LinearRegression()
    log_model.fit(X_log, Y)
    y_pred_log = log_model.predict(X_log)
    r2_log = r2_score(Y, y_pred_log)
    if r2_log > best_r2:
        best_r2 = r2_log
        best_equation = (lambda x:(log_model.coef_[0] * math.log(x) + log_model.intercept_), best_r2)
    
    return best_equation

In [28]:
def run_equation(row, predict_year):
    predict_year = predict_year - 2018
    given_values = [] 
    # largest year 
    for key in row.keys():
        given_values.append(row[key])
    """ 
    if predict_year in given_values:
        return
    """
    equation, equation_score = best_regression_model(deepcopy(row.keys().values), deepcopy(np.array(given_values)))
    if equation_score > 0.4:
        return(max(round(equation(predict_year)),0))
    else:
        # indexing using last 2 given, ignoring first
        print(round(np.mean(np.array(given_values[-2:]))))
        return round(np.mean(np.array(given_values[-2:])))

In [29]:
try:
    combined_df.drop("predict", axis = 1, inplace=True)
except:
    pass

In [30]:
combined_df

,0,1,2,3,4
74549,0.1,0.1,0.1,0.1,0.1
73847,0.1,0.1,0.1,0.1,0.1
73554,0.1,0.1,0.1,0.1,0.1
74748,0.1,0.1,0.1,0.1,0.1
73104,19.0,40.0,9.0,108.0,92.0
...,...,...,...,...,...
74759,0.1,0.1,0.1,0.1,0.1
74051,0.1,0.1,0.1,0.1,0.1
73735,0.1,0.1,0.1,91.0,0.1
73722,0.1,0.1,0.1,0.1,52.0


In [31]:
testing_df = combined_df.loc[["73007","73050", "74437"]]

In [32]:
testing_df["predict"] = testing_df.apply(run_equation, args = (2024,), axis = 1)

3


C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)


In [33]:
testing_df

,0,1,2,3,4,predict
73007,0.1,0.1,1.0,6.0,0.1,3
73050,6.0,7.0,3.0,0.1,0.1,0
74437,6.0,0.1,0.1,3.0,0.1,1


In [34]:
combined_df["predict"] = combined_df.apply(run_equation, args = (2023,), axis = 1)

C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\U

23
8
4
104
0
26


C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\U

88
121
0
0
0
0
13
7
760
54
0
0


C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\U

...

C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\U

0
53
0
0
242
0
10
7


C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\U

26
0
46


C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)
C:\Users\kapat\AppData\Local\Temp\ipykernel_19116\2472789310.py:39: RuntimeWarning: divide by zero encountered in log
  X_log = np.log(K.astype(float)).reshape(-1,1)


In [35]:
combined_df.loc["73007"]

0          0.1
1          0.1
2          1.0
3          6.0
4          0.1
predict    3.0
Name: 73007, dtype: float64

In [36]:
def convert_key_to_year(item):
    try: 
        year =  int(item)
        return year + 2018
    except:
        return item

In [39]:
combined_df.keys()

Index([0, 1, 2, 3, 4, 'predict'], dtype='object')

In [51]:
column_dict = {}
for key, value in zip(list(combined_df.keys()), list(map(convert_key_to_year, combined_df.keys()))):
    column_dict[key] = value

In [52]:
column_dict

{2018: 4036, 2019: 4037, 2020: 4038, 3: 2021, 4: 2022, 'predict': 'predict'}

In [53]:
combined_df.rename(column_dict, axis = 1, inplace=True)

In [54]:
combined_df

,4036,4037,4038,2021,2022,predict
74549,0.0,0.0,0.0,0.0,0.0,0
73847,0.0,0.0,0.0,0.0,0.0,0
73554,0.0,0.0,0.0,0.0,0.0,0
74748,0.0,0.0,0.0,0.0,0.0,0
73104,19.0,40.0,9.0,108.0,92.0,146
...,...,...,...,...,...,...
74759,0.0,0.0,0.0,0.0,0.0,0
74051,0.0,0.0,0.0,0.0,0.0,0
73735,0.0,0.0,0.0,91.0,0.0,46
73722,0.0,0.0,0.0,0.0,52.0,94


In [55]:
combined_df.replace(0.1, 0, inplace=True)

In [56]:
np.sum(combined_df["predict"])

np.int64(53668)

In [57]:
combined_df.to_csv("output2.csv")

In [58]:
easy = round(combined_df.sum())

In [59]:
easy.rename("Equipment Count")

4036       17568.0
4037        7465.0
4038       38987.0
2021       34285.0
2022       31642.0
predict    53668.0
Name: Equipment Count, dtype: float64

In [60]:
easy

4036       17568.0
4037        7465.0
4038       38987.0
2021       34285.0
2022       31642.0
predict    53668.0
dtype: float64